### **One-Way ANOVA TEST**


*The aim of this analysis is to statistically know if the proposed controller placementgorithm truely has a significant low output loss when it is compared with the reviewed Mopso, NSGA-II and NSGA-III algorithms. The importance of this analysi is to help to identify the most recommended controller placement alorithm with the least significant output loss in the SDN.* 

This link https://www.reneshbedre.com/blog/anova.html?utm_content=cmp-true explains more on the ANOVA test method.


In [ ]:
# we will use bioinfokit (v1.0.3 or later) for performing tukey HSD and some other tests
# check documentation here https://github.com/reneshbedre/bioinfokit
!pip install bioinfokit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Conduct random sampling
import pandas as pd
import numpy as np

input_value = input("Enter 1 to conduct sampling or 0 to use the documented 11 sample size dataset")
if (int(input_value) == 1):
  sample_size_input = input("Enter any value greater than 9:")
  if (int(sample_size_input) >= 10):  

      # Set the seed value for reproducibility
      np.random.seed(123)

      # load data file
      #df = pd.read_excel("final output loss.xlsx")
      df=pd.read_excel('/content/drive/MyDrive/final output loss.xlsx')

      #Sample the data:
      #Note: I decided not to use (sample = df.sample(n=sample_size, random_state=123, replace=False)) code 
      #becuase the independent sampling of each data column will reduce the sampling biasness.
      sample_size = int(sample_size_input)
      proposed  = np.random.choice(df['proposed'], sample_size, replace=False)
      mopso     = np.random.choice(df['mopso'], sample_size, replace=False)
      nsga2     = np.random.choice(df['nsga2'], sample_size, replace=False)
      nsga3     = np.random.choice(df['nsga3'], sample_size, replace=False)

      # Combine the vectors back into a DataFrame
      df = pd.DataFrame({'proposed': proposed, 'mopso': mopso, 'nsga2': nsga2, 'nsga3': nsga3})

      #write the new data to textfile and save it
      file_path = 'final output loss.csv'

      with open(file_path, 'w') as file:
          file.write(df['proposed'].to_string(index=False))
        
      # Print a confirmation message
      print(f"DataFrame written to file: {file_path}")
      print(df.tail(5))

  else:
    print("You entered the wrong value. The process will end now. Thnal you.")
elif (int(input_value) == 0):
  #df = pd.read_csv("final output loss.txt", sep="\t")
  df=pd.read_csv("/content/drive/MyDrive/final output loss.txt", sep="\t")
  print("The built-in data of sample size 11 had been stored in the memory now.")
  print(df)
else:
  #The process ends
  print("You entered the wrong ressponse. I am sorry that the process will be abordted now. \n Kindly run the programme again. Thank you")

Enter 1 to conduct sampling or 0 to use the documented 11 sample size dataset1
Enter any value greater than 9:150


FileNotFoundError: ignored

# **Data Importation**

*The network final output loss data for this analysis, with four columns, is importated below.*

In [ ]:
# load data file
#

# reshape the d dataframe suitable for statsmodels package 
df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['proposed', 'mopso', 'nsga2', 'nsga3'])

#print(df_melt)

# replace column names
df_melt.columns = ['index', 'treatments', 'value']

#print(df['proposed'])



---
## **The four controllers placement algorithms means**

*The network final output loss means for the four controllers placement algorithms (the Proposed algorithms, MOPSO, NSGA-II and NSGA-III algorithms) in the appropriate placement of controller in the SDN are:*

In [ ]:
print("The network final output loss means of the algorithms are: \n",df.mean())
print("\n ============================ \n")
print("The network final output loss variances of the algorithms are: \n",df.var())

# **Interpretation:**

This means that the proposed algorithm has the network final output loss means of 943.540408 while the MOPSO algorithms has the highest network final output loss means of 1459.861607. This study is interested to know if there is significant difference among these fours controller based on the network final output loss data. *


---


# **Box plot Presentation:**

*The Box plot is plotted below.*

In [ ]:
# generate a boxplot to see the data distribution by treatments. Using boxplot, we can 
# easily detect the differences between different treatments
import matplotlib.pyplot as plt
import seaborn as sns
ax = sns.boxplot(x='treatments', y='value', data=df_melt, color='#99c2a2')
ax = sns.swarmplot(x="treatments", y="value", data=df_melt, color='#7d0013')
plt.show()


**Interpretation of the Box plot for the four controllers:**

Each box represents each controller. There seems to be complications on thise plot. I hope to inestigate further and later.

---



# **One Way ANOVA Test**

*Below id the one-way anova test analysis and results.*

H0: There are no significant differences among final network loss mean produced by the four algorithms at 5% significant level.

H1: At least, one of the algorithms has final network loss mean that is different from others at 5% significant level.

In [ ]:
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(df['proposed'], df['mopso'], df['nsga2'], df['nsga3'])
print(fvalue, pvalue)

print()
if (pvalue < 0.05):
  print(" Decision: Reject the Null hypothesis \n At least, one of the algorithms has final network loss mean that is different from others at 5% significant level.")
else:
  print(" Decision: Accept the Null hypothesis \n There are no significant differences among final network loss mean produced by the four algorithms at 5% significant level.")

print()
print("==================================================================")

# get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Ordinary Least Squares (OLS) model
model = ols('value ~ C(treatments)', data=df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

print()
print("==================================================================")

# ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=df_melt, res_var='value', anova_model='value ~ C(treatments)')
res.anova_summary

**ANOVA TEst Result Interpretation**
*The p value obtained from ANOVA analysis is significant (p < 0.05), and therefore, we conclude that there are significant differences among treatments.*


---



# **Post HOC TesT**

*Below is Post- HOC Test using Tukey's HSD test.*

In [ ]:
from bioinfokit.analys import stat
# perform multiple pairwise comparison (Tukey's HSD)
# unequal sample size data, tukey_hsd uses Tukey-Kramer test
res = stat()
res.tukey_hsd(df=df_melt, res_var='value', xfac_var='treatments', anova_model='value ~ C(treatments)')
res.tukey_summary

**Interpretation of Post-HOC:** *Above results from Tukey’s HSD suggests that except A-C, all other pairwise comparisons for controller location means rejects null hypothesis (p < 0.05) and indicates statistical significant differences between the pair of final output loss.*

---

## **Normal distribution:**



In [ ]:
#Test ANOVA assumptionsPermalink
#ANOVA assumptions can be checked using test statistics (e.g. Shapiro-Wilk, Bartlett’s, Levene’s test, Brown-Forsythe test) 
#and the visual approaches such as residual plots (e.g. QQ-plots) and histograms.

#The visual approaches perform better than statistical tests. For example, the Shapiro-Wilk test has low power for small 
#sample size data and deviates significantly from normality for large sample sizes (say n > 50). For large sample sizes, you 
#should consider to use QQ-plot for normality assumption.

# QQ-plot
import statsmodels.api as sm
import matplotlib.pyplot as plt
# res.anova_std_residuals are standardized residuals obtained from ANOVA (check above)
sm.qqplot(res.anova_std_residuals, line='45')
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Standardized Residuals")
plt.show()

# histogram
plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
plt.xlabel("Residuals")
plt.ylabel('Frequency')
plt.show()

**Interpretation of standardized residuals plot:**
*As the standardized residuals lie closely around the 45-degree line, it suggests that the residuals are are likely to be approximately normally distributed.*

**Interpretation of histogram plot:**
In the histogram, the distribution looks approximately left tailed and suggests that residuals are not like to be approximately normally distributed. 

The normal distrinution and homogenuity teste will be conducted for confirmation of what we suspected in the graph.

---

## **Shapiro-Wilk test for Normality assumption test:**

H0: The dataset is drawn from normal distribution at 5% significant level.

H1: The dataset is not drawn from normal distribution at 5% significant level.

In [ ]:
#Shapiro-Wilk test can be used to check the normal distribution of residuals. Null hypothesis: data is drawn from normal distribution.

import scipy.stats as stats
w, pvalue = stats.shapiro(model.resid)
print(w, pvalue)

print("==============================")

if (pvalue > 0.05):
  print(" Decision: Accept the Null hypothesis \n Interpretation: The data is drawn from normal distribution at 5% significant level")
else:
  print(" Decision: Reject the Null hypothesis \n Interpretation: The data is NOT drawn from normal distribution at 5% significant level")

**Interpretation Shapiro-Wilk test:**
*As the p value (0.6609 > 0.05) is significant, we accept the null hypothesis and conclude that data is drawn from normal distribution.*



---



## **Homogeneity of variances test:**

**H0:** The four Samples, from the populations, have equal variances.

**H1:** The four Samples, from the populations, do not have equal variances.

# **Bartlett's test**
*Below is bartlett's test for equality of variance test.*

In [ ]:
import scipy.stats as stats
w, pvalue = stats.bartlett(df['proposed'], df['mopso'], df['nsga2'], df['nsga3'])
print(w, pvalue)

# if you have a stacked table, you can use bioinfokit v1.0.3 or later for the bartlett's test
from bioinfokit.analys import stat 
res = stat()
res.bartlett(df=df_melt, res_var='value', xfac_var='treatments')
res.bartlett_summary
#res.bartlett_summary.values

# **Levene's test**
*Below is levene's test for equality of variance test.*

In [ ]:
# if you have a stacked table, you can use bioinfokit v1.0.3 or later for the Levene's test
from bioinfokit.analys import stat 
res = stat()
res.levene(df=df_melt, res_var='value', xfac_var='treatments')
res.levene_summary

**Interpretation:**

*As the p value, for both **Bartlett's test (0.0011 < 0.05) and levene test (0.0130 < 0.05)** is significant, we reject the null hypothesis and conclude that four algorithms do not have equal variances.*


---



## **Kruskal-Wallis test**

This test replaces the ANOVA test dues to the violation of the homoscedastisity of variance assumption.

HO: There is no significant differences among network final output loss medians for the four algorithms at 5% significant level.

H1: At least one of the algorithms produces the network final output loss median that is significantly different from other algorithms at 5% significant level.



In [ ]:
from scipy.stats import kruskal

# Separate the data into the groups based on the fourth column
proposed_algorithm = df['proposed']
mopso_algorithm = df['mopso']
nsga2_algorithm = df['nsga2']
nsga3_algorithm = df['nsga3']

# Conduct the Kruskal-Wallis test on the groups
stat, p = kruskal(proposed_algorithm, mopso_algorithm, nsga2_algorithm, nsga3_algorithm)

# Print the results
print('Kruskal-Wallis Test')
print('-------------------')
print('H statistic =', stat)
print('p value =', p)

print()

# If the p-value is less than the significance level (usually 0.05), reject the null hypothesis
if p < 0.05:
    print(' Decision: There is significant evidence to reject the null hypothesis at 5% significant level. \n Conclusion: At least one of the algorithms produces the network final output loss median that is significantly different from other algorithms at 5% significant level')
else:
    print('There is not enough evidence to reject the null hypothesis at 5% significant level. \n Conclusion: There is no significant differences among network final output loss medians for the four algorithms at 5% significant level.')

**Since there is evidence that at least one of the algorithms produces the network final output loss median that is significantly different from other algorithms at 5% significant level, hence the test will compare the lagorithms pairwisely using the Pairwise Wilcoxon rank-sum test.**


---



## **Pairwise Wilcoxon rank-sum test**

In [ ]:
from scipy.stats import ranksums

# data for groups A, B, and C
print("The median will be compared for difference. The median for the final output loss for:")
print("Proposed Algorithm   :", df['proposed'].median())
print("Mopso Algorithm      :", df['mopso'].median())
print("NSGA-II Algorithm    :", df['nsga2'].median())
print("NSGA-III Algorithm   :", df['nsga3'].median())
print()

# pairwise comparisons of groups
print("Proposed Algorithm and Mopso Algorithm Comparism")
print("================================================")
print(ranksums(proposed_algorithm, mopso_algorithm))
pvalue1 = ranksums(proposed_algorithm, mopso_algorithm).pvalue
if (pvalue1 < 0.05):
  print("The null hypothesis is rejected. It is concluded that there is significant difference \n between the two median values of the two compared Algorithms at 5% signifiant level. \n This implies that the proposed algorithm has significant low final output loss than the Mopso algorithm at 5% significant level.")
else:
  print("The null hypothesis is accepted. It is concluded that there is no significant difference \n between the two median values of the compared two Algorithms at 5% signifiant level. \n This implies that any obseved difference between the medians of the two compared algorithms is not significant at 5% significant level.")

print()
print("Proposed Algorithm and NSGA-II Algorithm Comparism")
print("================================================")
print(ranksums(proposed_algorithm, nsga2_algorithm))
pvalue2 = ranksums(proposed_algorithm, nsga2_algorithm).pvalue
if (pvalue2 < 0.05):
  print("The null hypothesis is rejected. It is concluded that there is significant difference \n between the two median values of the two compared Algorithms at 5% signifiant level. \n This implies that the proposed algorithm has significant low final output loss than the Mopso algorithm at 5% significant level.")
else:
  print("The null hypothesis is accepted. It is concluded that there is no significant difference \n between the two median values of the compared two Algorithms at 5% signifiant level. \n This implies that any obseved difference between the medians of the two compared algorithms is not significant at 5% significant level.")

print()
print("Proposed Algorithm and NSGA-III Algorithm Comparism")
print("================================================")
print(ranksums(proposed_algorithm, nsga3_algorithm))
pvalue3 = ranksums(proposed_algorithm, nsga3_algorithm).pvalue
if (pvalue3 < 0.05):
  print("The null hypothesis is rejected. It is concluded that there is significant difference \n between the two median values of the two compared Algorithms at 5% signifiant level. \n This implies that the proposed algorithm has significant low final output loss than the Mopso algorithm at 5% significant level.")
else:
  print("The null hypothesis is accepted. It is concluded that there is no significant difference \n between the two median values of the compared two Algorithms at 5% signifiant level. \n This implies that any obseved difference between the medians of the two compared algorithms is not significant at 5% significant level.")

print()
print("NSGA-II Algorithm and NSGA-III Algorithm Comparism")
print("================================================")
print(ranksums(nsga2_algorithm, nsga3_algorithm))
pvalue4 = ranksums(nsga2_algorithm, nsga3_algorithm).pvalue
if (pvalue4 < 0.05):
  print("The null hypothesis is rejected. It is concluded that there is significant difference \n between the two median values of the two compared Algorithms at 5% signifiant level. \n This implies that the proposed algorithm has significant low final output loss than the Mopso algorithm at 5% significant level.")
else:
  print("The null hypothesis is accepted. It is concluded that there is no significant difference \n between the two median values of the compared two Algorithms at 5% signifiant level. \n This implies that any obseved difference between the medians of the two compared algorithms is not significant at 5% significant level.")

print()
print("Mopso Algorithm and NSGA-III Algorithm Comparism")
print("================================================")
print(ranksums(mopso_algorithm, nsga3_algorithm))
pvalue5 = ranksums(mopso_algorithm, nsga3_algorithm).pvalue
if (pvalue5 < 0.05):
  print("The null hypothesis is rejected. It is concluded that there is significant difference \n between the two median values of the two compared Algorithms at 5% signifiant level. \n This implies that the proposed algorithm has significant low final output loss than the Mopso algorithm at 5% significant level.")
else:
  print("The null hypothesis is accepted. It is concluded that there is no significant difference \n between the two median values of the compared two Algorithms at 5% signifiant level. \n This implies that any obseved difference between the medians of the two compared algorithms is not significant at 5% significant level.")

print()
print("Mopso Algorithm and NSGA-II Algorithm Comparism")
print("================================================")
print(ranksums(mopso_algorithm, nsga2_algorithm))
pvalue6 = ranksums(mopso_algorithm, nsga2_algorithm).pvalue
if (pvalue6 < 0.05):
  print("The null hypothesis is rejected. It is concluded that there is significant difference \n between the two median values of the two compared Algorithms at 5% signifiant level. \n This implies that the proposed algorithm has significant low final output loss than the Mopso algorithm at 5% significant level.")
else:
  print("The null hypothesis is accepted. It is concluded that there is no significant difference \n between the two median values of the compared two Algorithms at 5% signifiant level. \n This implies that any obseved difference between the medians of the two compared algorithms is not significant at 5% significant level.")
